В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [1]:
import numpy as np
from scipy import stats

def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )


def proportions_diff_z_test(z_stat, alternative='two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)
    
    
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))


def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)



a = np.array([1 if i < 10 else 0 for i in range(34)])
b = np.array([1 if i < 4 else 0 for i in range(16)])
z_stat = proportions_diff_z_stat_ind(a, b)
print(z_stat)
print(round(proportions_diff_z_test(z_stat, 'greater'), 4))

0.32410186177608225
0.3729


Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

логистическая регрессия по признакам X_1,X_2,X_3

логистическая регрессия по признакам X_4,X_5,X_6

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили 5.5\times10^{-8}5.5×10 нужно ввести 8).

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd


model_1 = LogisticRegression()
model_2 = LogisticRegression()

data = pd.read_table('banknotes.txt')
data_train, data_test = train_test_split(data, test_size=50, random_state=1)
data_train_1 = data_train[['X1', 'X2', 'X3']]
data_train_2 = data_train[['X4', 'X5', 'X6']]
data_test_1 = data_test[['X1', 'X2', 'X3']]
data_test_2 = data_test[['X4', 'X5', 'X6']]
y_train = data_train['real']
y_test = data_test['real']
data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [3]:
model_1.fit(data_train_1, y_train)
model_2.fit(data_train_2, y_train)
preds_1 = [1 if x == y else 0 for x, y in zip(model_1.predict(data_test_1), y_test)]
preds_2 = [1 if x == y else 0 for x, y in zip(model_2.predict(data_test_2), y_test)]
z_stat = proportions_diff_z_stat_ind(preds_1, preds_2)
print(z_stat)
p = stats.norm.cdf(z_stat)
print(p)

-2.8764124744526685
0.002011118636027643


In [4]:
print(proportions_diff_confint_ind(a, b))

(-0.2175577216559601, 0.3057930157736072)


Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [5]:
mean_1 = 525
std_1 = 100
mean_2 = 541.4
n = 100

z_stat = float((mean_2 - mean_1) / (std_1 / np.sqrt(n)))
print(z_stat)
p = stats.norm.cdf(z_stat)
print(round(1 - p, 4))

1.6399999999999977
0.0505


In [6]:
new_mean = 541.5

z_stat_2 = float((new_mean - mean_1) / (std_1 / np.sqrt(n)))
print(z_stat_2)
p_2 = stats.norm.cdf(z_stat_2)
print(round(1 - p_2, 4))

1.65
0.0495
